In [1]:
!pip install import_ipynb
!pip install tensorflow-datasets

In [4]:
 # .py로 매번 수정하기 귀찮아서 ipynb 파일로 import가능하게 만드는 라이브러리
import import_ipynb 

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import models


GoogLeNet = models.InceptionNet

tfds.list_builders()[:-1]

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'booksum',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19',
 'covid19sum',
 'crema_d',
 'cs_restaurants',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'd4rl_adroit_door',
 'd4rl_adr

In [5]:
#하이퍼 파라미터
batch_size = 128
num_epoch = 100
train_accuracies = []
test_accuracies = []

def normalize(dataset):    
    image, label = tf.cast(dataset['image'], tf.float32) / 255.0, dataset['label']
    return image, label

# 데이터 생성
dataset, info = tfds.load('resisc45', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']

# open_images_dataset = tfds.image.OpenImagesV4()
# open_images_dataset.download_and_prepare(download_dir="/notebooks/dataset/")
#ds = tf.load('open_images_v4', data_dir='/notebooks/open_images_dataset/extracted', download=False)


AssertionError: Manual directory C:\Users\sangt\tensorflow_datasets\downloads\manual does not exist or is empty. Create it and download/extract dataset artifacts in there using instructions:
Dataset can be downloaded from OneDrive:
https://1drv.ms/u/s!AmgKYzARBl5ca3HNaHIlzp_IXjs
After downloading the rar file, please extract it to the manual_dir.

In [ ]:

dataset_train = dataset_train.map(normalize).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(normalize).batch(batch_size)

datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, zoom_range=0.1)

# GoogleNet 객체 생성
model = GoogLeNet((256, 256, 3), 10)
model.build(input_shape = (None, 256, 256, 3))
model.summary()

# Prepare Taining 라벨링

############## #사용자 루프 모델 학습 ######
# model.compile() 기능을 사용자가 직접 수동으로 하는 것
#def compile_by_users():
    
for image, label in dataset_train.take(1):
    fig, axes = plt.subplots(8, 4)
    fig.set_size_inches(10, 16)
    for i in range(32):
        axes[i//4, i%4].imshow(image[i])
        axes[i//4, i%4].axis('off')
        axes[i//4, i%4].set_title(label_map[label[i].numpy()], fontsize=15)
    plt.show()

In [ ]:
# 입력값에 대해 원핫라벨링한 뒤, 교차 엔트로피에 통과하여 loss를 검사
loss_object = keras.losses.SparseCategoricalCrossentropy()
# 최적화는 adam
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name = 'train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = keras.metrics.Mean(name = 'test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')
    


#compile_by_users()

def train_step(image, label): # 훈련 이미지, 훈련 레이블
    with tf.GradientTape() as tape:
        
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
    
def test_step(image, label): # 훈련 이미지, 훈련 레이블
        
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)
    

In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
for epoch in range(num_epoch):
    i = 0
    for image, label in dataset_train:
        for _image, _label in datagen.flow(image, label, batch_size = batch_size):
            printProgressBar(i + 1, len(dataset_train),  prefix = 'Progress:', suffix = 'Complete', length = 50)
            # Data Augmentabtion된 훈련 데이터를 _image, 훈련 레이블을 _label로 저장
            # Data Augmentabtion이란 데이터셋을 여러 방법으로 변경하여 데이터셋 개수를 늘리는 방법이다.
            # 회전을 시킨다거나, 임의의 범위를 잘라낸다건, 하나의 소르를 가지고 데이터를 우려내는 방법이다.
            train_step(_image, _label) # 모델에 data Augmentation된 훈련 데이터셋을 넣음
            i+=1
            break
    # 훈련 데이터 셋에 대한 학습 종료

    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        # 시험 데이터 셋 테스트 종료
        
    train_accuracies.append(train_accuracy.result())
    test_accuracies.append(test_accuracy.result())
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100,
                          test_loss.result(), test_accuracy.result() * 100))